In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_excel('../Input/DadosCompeticao.xlsx')

#### Pytorch LSTM -> WRMSE = 0.09791456241015854

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
N_PAST = min(10, df.shape[0] - N_FUTURE - 1)
N_FUTURE = 12
EPOCHS = 100
LR = 0.01 # Validar Learning rate

## ========== Aplicar grid search para hiperparametros ============

In [ ]:
def create_dataset(series, n_past, n_future):
    X, y = [], []
    for i in range(len(series) - n_past - n_future + 1):
        X.append(series[i:i + n_past])
        y.append(series[i + n_past:i + n_past + n_future])
    return np.array(X), np.array(y)

In [ ]:
class LSTMModel(nn.Module): #Todas apresentam aplicação linear, validar para tanh ou sigmoid
    def __init__(self, input_size=1, hidden_size=64, num_layers=4, output_size=N_FUTURE, dropout=0.3):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0.0,
            batch_first=True
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        x = self.dropout(lstm_out[:, -1, :])  # pega o último estado oculto -> neurônio 64
        return self.fc(x)


In [ ]:
forecasts = {}
scores = {}

for col in df.columns:
    print(f"\n ==== Treinando {col} ====")

    serie = df[col].values.reshape(-1, 1)
    scaler = MinMaxScaler()
    serie_scaled = scaler.fit_transform(serie)

    X_np, y_np = create_dataset(serie_scaled, N_PAST, N_FUTURE)

    if len(X_np) == 0:
        print(f"Série {col} não há valores suficientes para predição")
        continue

    X_tensor = torch.tensor(X_np, dtype=torch.float32).to(DEVICE)
    y_tensor = torch.tensor(y_np, dtype=torch.float32).to(DEVICE)

    model = LSTMModel().to(DEVICE) ## GPU on
    criterion = nn.MSELoss() # priorizar erro quadrádico médio -> Erro raíz
    optimizer = torch.optim.Adam(model.parameters(), lr=LR) # Otimizador Adam

    for epoch in range(EPOCHS):
        model.train()
        optimizer.zero_grad()
        output = model(X_tensor)
        loss = criterion(output, y_tensor.squeeze(-1))
        loss.backward()
        optimizer.step()

    model.eval()
    last_input = torch.tensor(serie_scaled[-N_PAST:], dtype=torch.float32).view(1, N_PAST, 1).to(DEVICE) #transform para tensor

    with torch.no_grad():
        prediction_scaled = model(last_input).cpu().numpy().flatten()

    prediction = scaler.inverse_transform(prediction_scaled.reshape(-1, 1)).flatten()
    true_values = df[col].values[-N_FUTURE:]

    if len(true_values) == N_FUTURE:
        rmse = np.sqrt(mean_squared_error(true_values, prediction))
        scores[col] = rmse
        forecasts[col] = prediction
    else:
        scores[col] = np.nan
        forecasts[col] = prediction


In [ ]:
df_forecasts = pd.DataFrame(forecasts)
df_scores = pd.DataFrame(list(scores.items()), columns=['Série', 'RMSE'])

In [ ]:
df_forecasts.to_excel('../Output/v4/previsoes_torch.xlsx', index=False)
df_scores.to_excel('../Output/v4/scores_torch.xlsx')

In [ ]:
wrmse = lambda rmse: (rmse[:11] * (1 / 11)).sum()

In [41]:
print(f'WRMSE = {wrmse(df_scores["RMSE"])}')

WRMSE = 0.09791456241015854
